<a href="https://colab.research.google.com/github/dnguyend/Manifolds.jl/blob/master/misc_stiefel_frechet/StiefelFrechetLogJulia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proposal to incorporate the algorithm to compute Riemannian logarithm on Stiefel manifold with submersion (deformation) metrics using Frechet derivatives.
* The metric family includes both embedded and canonical metrics.

* We [implement](https://github.com/dnguyend/Manifolds.jl/blob/master/src/expm_frechet.jl) Frechet derivatives for the matrix exponential in Julia (not available in LinearAlgebra) - porting the function expm_frechet_algo_64 in scipy.linalg. This function is of general interest so probably should be included in the LinearAlgbra package (raised in [LinearAlgebra, issue 5840](https://github.com/JuliaLang/julia/issues/5840) ). 
* Our version of Riemannian logarithm [is implemented](https://github.com/dnguyend/Manifolds.jl/blob/master/src/manifolds/StiefelFrechet.jl) as the method *log_lbfgs*. We compute the exact gradient of the cost function, the square Euclidean distance of the geodesic end point $Y(1)$ to the tartget point $q$, with $Y(0) = p$. The gradient is expressed using Frechet derivatives. The algorithm calls the LBFGS optimizer in Optim (new dependency is required). This is an alternative to the function *log* in StiefelSubmersionMetric.jl

* One modification versus the method described in [[1]](https://link.springer.com/article/10.1007/s10957-022-02012-3) is we call LBFGS to optimize up to a distance *pretol*. After that we just run a simple descent based on the distance. This modification help speed up the algorithm.
* The new algorithm compares well with the log function for StiefelSubmersionMetric.
* We also implemented the geodesic formula 3.2 in [[1]](https://link.springer.com/article/10.1007/s10957-022-02012-3) in function 
# More details:
* For expm_frechet
 * in [src/expm_frechet.jl](https://github.com/dnguyend/Manifolds.jl/blob/master/src/expm_frechet.jl)
 * We verified numerically the result, using the definition of Frechet derivative as a directional derivative, and compare with scipy. 
 * We implement a version with pre-allocated memory (*expm_frechet!*) useful in a loop where the Frechet derivative is called repeatedly. A simple version (*expm_frechet*) is also implemented.
 * Timewise, *expm_frechet!* executes typically 3 times for less than the execution time for $exp$, while *expm_frechet* takes between 3 to 4 times.
* For log_lbfgs:
  * In [src/manifolds/StiefelFrechet.jl](https://github.com/dnguyend/Manifolds.jl/blob/master/src/manifolds/StiefelFrechet.jl)
  * For longer distance, a pretol of 1e-5 produces better result.
  * When data is generated using a higher intial velocity, geodesic distance could be shorter than the data generating vector.

[1] Nguyen, D. Closed-form Geodesics and Optimization for Riemannian Logarithms of Stiefel and Flag Manifolds. J Optim Theory Appl 194, 142–166 (2022). https://doi.org/10.1007/s10957-022-02012-3



## The workbook could be viewed as is, but could also be run in a colab environment.
## Instructions to run the workbook

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab 

1. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). **This takes a couple of minutes.**
2. **Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.**

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 1 and 2.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 1 and 2.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-10-23 21:35:33 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Success! Please reload this page and jump to the next section.


# Checking the Installation
**REMEMBER TO RELOAD THE PAGE BY RUNNING F5 IF the following command does not work**

The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 2


In [2]:
using BenchmarkTools
using LinearAlgebra

Load the forked version of Manifolds.jl

In [3]:
using Pkg
Pkg.add(url="https://github.com/dnguyend/Manifolds.jl")

     Cloning git-repo `https://github.com/dnguyend/Manifolds.jl`
    Updating git-repo `https://github.com/dnguyend/Manifolds.jl`
    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed ZygoteRules ──────────────────── v0.2.2
   Installed HypergeometricFunctions ──────── v0.3.11
   Installed SimpleTraits ─────────────────── v0.9.4
   Installed FiniteDiff ───────────────────── v2.15.0
   Installed DiffRules ────────────────────── v1.12.0
   Installed RecursiveArrayTools ──────────── v2.32.0
   Installed StaticArrays ─────────────────── v1.5.9
   Installed Calculus ─────────────────────── v0.5.1
   Installed IteratorInterfaceExtensions ──── v1.0.0
   Installed Inflate ──────────────────────── v0.1.3
   Installed GPUArraysCore ────────────────── v0.1.2
   Installed ArnoldiMethod ────────────────── v0.2.0
   Installed ManifoldsBase ────────────────── v0.13.22
   Installed StatsFuns ────────────────────── v1.0.1
   Installed DataValueInterfaces ─

# Test expm_frechet! and expm_frechet

In [9]:
using Manifolds
using Printf
using Random

function linf(mat)
  return maximum(abs.(mat))
end  

Random.seed!(0)

dlt = 1e-7
function test_one(n, Anorm)
  buff = Array{Float64, 2}(undef, 16*n, n)

  # set up view - for expm_frechet!, first 2 blocks of buff are the returns, exp(A) and the Frechet derivative
  # $\lim_{t\to 0}\frac{1}{t}(exp(A+ tE) - exp(A))$
  @views begin
      expA = buff[1:n, :]
      expAE = buff[n+1:2*n, :]
  end

  ft = (rand() + .1)*Anorm/1.1
  A = rand(n, n)

  A = A / norm(A, 2)*ft
  E = rand(n, n)
  E = E / norm(E, 2)*ft

  Manifolds.expm_frechet!(buff, A, E)
  expA1, expAE1 = Manifolds.expm_frechet(A, E)


  @printf "compare with LinearAlgebra diff=%.5e\n" linf(expA - exp(A))
  @printf "compare 2 versions exp(A) diff=%.5e\n" linf(expA1 - expA)
  @printf "compare 2 versions exp(A) diff=%.5e\n" linf(expAE1 - expAE)
  @printf "compare numerical derivative diff=%.5e\n" linf((exp(A+dlt*E) - exp(A))/dlt - expAE)  
end
test_one(3, 10)
test_one(100, 4)

compare with LinearAlgebra diff=1.59162e-12
compare 2 versions exp(A) diff=0.00000e+00
compare 2 versions exp(A) diff=0.00000e+00
compare numerical derivative diff=1.13569e-03
compare with LinearAlgebra diff=4.44089e-16
compare 2 versions exp(A) diff=0.00000e+00
compare 2 versions exp(A) diff=0.00000e+00
compare numerical derivative diff=7.06184e-08


* Benchmark

In [10]:
display(@benchmark exp(A))
display(@benchmark Manifolds.expm_frechet(A, E))
display(@benchmark Manifolds.expm_frechet!(buff, A, E))

BenchmarkTools.Trial: 3220 samples with 1 evaluation.
 Range (min … max):  1.197 ms … 24.276 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     1.298 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.530 ms ±  1.208 ms  ┊ GC (mean ± σ):  9.91% ± 12.75%

  █▇▄▂                                                       ▁
  █████▆▅▃▄▃▅▅▃▁▅▄▄▁▄▃▃▃▅▃▁▅▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▆█▇▇▆▄▁▁▃▅ █
  1.2 ms       Histogram: log(frequency) by time     7.34 ms <

 Memory estimate: 1.22 MiB, allocs estimate: 35.

BenchmarkTools.Trial: 1093 samples with 1 evaluation.
 Range (min … max):  3.092 ms … 23.965 ms  ┊ GC (min … max):  0.00% … 24.15%
 Time  (median):     3.653 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.526 ms ±  2.379 ms  ┊ GC (mean ± σ):  16.12% ± 19.62%

  █▅▅▇▆▄▄▃▁                          ▁▂▂         ▁            
  █████████▇█▅▅▄▅▄▁▄▅▁▁▁▅▄▅▁▁▁▁▁▄▁▁▆████▇▆▇▄▄▄▅▇▇█▇▇▅▄▁▅▁▁▁▅ █
  3.09 ms      Histogram: log(frequency) by time     11.9 ms <

 Memory estimate: 5.96 MiB, allocs estimate: 181.

BenchmarkTools.Trial: 1137 samples with 1 evaluation.
 Range (min … max):  3.320 ms … 19.945 ms  ┊ GC (min … max): 0.00% … 52.82%
 Time  (median):     3.627 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.354 ms ±  2.092 ms  ┊ GC (mean ± σ):  8.41% ± 14.15%

  ███▆▅▃▁▁                                     ▁▁             
  ████████▇▆▅▁▄▁▁▅▄▄▄▁▄▄▁▄▁▁▁▄▁▁▁▆▇▆▆█▇▇▆▁▇▆▇▇▇███▆▇▅▅▄▆▅▅▁▆ █
  3.32 ms      Histogram: log(frequency) by time     10.7 ms <

 Memory estimate: 2.83 MiB, allocs estimate: 91.

We also dump expm_frechet for 500 samples of matrices and compare with python in folder [test_expm_frechet](https://github.com/dnguyend/Manifolds.jl/tree/master/misc_stiefel_frechet/test_expm_frechet), a simple [python script](https://github.com/dnguyend/Manifolds.jl/blob/master/misc_stiefel_frechet/check_expm.py) read the jld files and compare with scipy.linalg.expm_frechet. Discrepency with Python is in the order of 1e-11.

## Riemannian Logarithm
* We implement the function Manifolds.log_lbfgs, structured after Manifolds.log
* A warmup run:

In [11]:
function randpoint(M)
    return project(M, randn(representation_size(M)))
end

function randvec(M, p)
    # generate unit vector
    X = project(M, p, randn(representation_size(M)))
    X ./= sqrt(inner(M, p, X, X))
    return X
end

# do a warm up run
n = 5
k = 3
α = .2

M = MetricManifold(Stiefel(n, k), StiefelSubmersionMetric(α))
p = randpoint(M)
X = randvec(M, p)

q = exp(M, p, X)

XA = Manifolds.log(M, p, q)
XB = Manifolds.log_lbfgs(M, p, q, tolerance=1e-10)
display(linf(XA - XB))

2.0138888473519856e-9

Now compare and benchmark for a bigger example. Generated distance is $0.5\pi$


In [12]:
 n = 1000
 k = 200

 # We test for all α >-1.
 α = 3*rand() - .9
 
 M = MetricManifold(Stiefel(n, k), StiefelSubmersionMetric(α))
 p = randpoint(M)
 X = randvec(M, p)

 # example with distance 0.5pi
 ft = .5
 q = exp(M, p, ft*pi*X)

 XOld = Manifolds.log(M, p, q)
 XF = Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, max_itr=1000, pretol=1e-3)
 display(linf(XOld - XF))
 display(linf(XOld - ft*pi*X))
 display(linf(XF - ft*pi*X))

8.274858229184723e-12

9.03289682635533e-12

7.580385971706072e-13

In [13]:
 display(@benchmark Manifolds.log(M, p, q))
 display(@benchmark Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, max_itr=1000, pretol=1e-3))

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  1.089 s …   1.211 s  ┊ GC (min … max): 6.01% … 8.87%
 Time  (median):     1.105 s              ┊ GC (median):    6.99%
 Time  (mean ± σ):   1.125 s ± 48.856 ms  ┊ GC (mean ± σ):  7.20% ± 1.04%

  █    █ █      █                                         █  
  █▁▁▁▁█▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.09 s         Histogram: frequency by time        1.21 s <

 Memory estimate: 725.86 MiB, allocs estimate: 3482.

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  756.746 ms …    1.184 s  ┊ GC (min … max):  8.28% … 37.72%
 Time  (median):     776.418 ms               ┊ GC (median):     8.30%
 Time  (mean ± σ):   846.239 ms ± 167.449 ms  ┊ GC (mean ± σ):  15.35% ± 12.26%

  █▁  ▁    ▁                                                  ▁  
  ██▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  757 ms           Histogram: frequency by time          1.18 s <

 Memory estimate: 419.68 MiB, allocs estimate: 3076.

* Another example. Generated distance is $2\pi$

In [14]:
 Random.seed!(1)
 ft = 2.
 q = exp(M, p, ft*pi*X)

 XOld = Manifolds.log(M, p, q)
 XF = Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, pretol=1e-5)
 display(linf(XOld - XF))
 display(linf(XOld - ft*pi*X))
 display(linf(XF - ft*pi*X))
 display(@benchmark Manifolds.log(M, p, q))
 display(@benchmark Manifolds.log_lbfgs(M, p, q, tolerance=1e-10, pretol=1e-5))

2.1575043834820562e-11

2.4556787159291105e-11

1.3750237060072834e-11

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.928 s …  2.932 s  ┊ GC (min … max): 6.61% … 7.89%
 Time  (median):     2.930 s             ┊ GC (median):    7.25%
 Time  (mean ± σ):   2.930 s ± 2.580 ms  ┊ GC (mean ± σ):  7.25% ± 0.91%

  █                                                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.93 s        Histogram: frequency by time        2.93 s <

 Memory estimate: 1.34 GiB, allocs estimate: 7304.

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.930 s …    2.462 s  ┊ GC (min … max):  5.02% … 19.15%
 Time  (median):     1.990 s               ┊ GC (median):     5.91%
 Time  (mean ± σ):   2.127 s ± 291.320 ms  ┊ GC (mean ± σ):  10.75% ±  7.91%

  █     █                                                  █  
  █▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.93 s         Histogram: frequency by time         2.46 s <

 Memory estimate: 862.55 MiB, allocs estimate: 9918.

*  We test extensively using the script [test_log_frechet.jl](https://raw.githubusercontent.com/dnguyend/Manifolds.jl/master/misc_stiefel_frechet/test_log_frechet.jl) to generate manifolds of different size, alpha and distance. We save the result in csv files, max_05 and max_05_pretol5 corresponding to distance at most $0.5\pi$, max_20 corresponds to distance at most $2\pi$ and max_40 corresponds to distance at most $4\pi$
We put data in data frames and summarize the results:

In [15]:
download("https://raw.githubusercontent.com/dnguyend/Manifolds.jl/master/misc_stiefel_frechet/max_05.csv", "max_05.csv")
download("https://raw.githubusercontent.com/dnguyend/Manifolds.jl/master/misc_stiefel_frechet/max_05_pretol5.csv", "max_05_pretol5.csv")
download("https://raw.githubusercontent.com/dnguyend/Manifolds.jl/master/misc_stiefel_frechet/max_20.csv", "max_20.csv")
download("raw.githubusercontent.com/dnguyend/Manifolds.jl/master/misc_stiefel_frechet/max_40.csv", "max_40.csv")

"max_40.csv"

In [16]:
Pkg.add("DataFrames")
Pkg.add("CSV")
using DataFrames, CSV

   Resolving package versions...
   Installed Crayons ──────────── v4.1.1
   Installed PooledArrays ─────── v1.4.2
   Installed InvertedIndices ──── v1.1.0
   Installed StringManipulation ─ v0.3.0
   Installed DataFrames ───────── v1.4.1
   Installed PrettyTables ─────── v2.1.2
    Updating `~/.julia/environments/v1.6/Project.toml`
  [a93c6f00] + DataFrames v1.4.1
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [a8cc5b0e] + Crayons v4.1.1
  [a93c6f00] + DataFrames v1.4.1
  [41ab1584] + InvertedIndices v1.1.0
  [2dfb63ee] + PooledArrays v1.4.2
  [08abe8d2] + PrettyTables v2.1.2
  [892a3eda] + StringManipulation v0.3.0
Precompiling project...
  ✓ InvertedIndices
  ✓ PooledArrays
  ✓ Crayons
  ✓ StringManipulation
  ✓ PrettyTables
  ✓ DataFrames
6 dependencies successfully precompiled in 44 seconds (192 already precompiled)
   Resolving package versions...
   Installed SentinelArrays ─ v1.3.16
   Installed InlineStrings ── v1.2.2
   Installed FilePathsBase ── v0.9.20
   Installe

* Comparing data, errOld is error using the old log, errNew is using the new log, timeOld is time using the old log, time new using new log. We also compare the norm. We will see sometime the log is better than the data generating norm (colum normGEN, ie if we generate point q using vector  $f\pi X$, old norm is $fp\pi |X|_{Riemann}$.

In [17]:
# at distance at most 0.5pi
 header = vec(["n", "k",	"alpha",	"errOld",	"errNew",	"timeOld",	"timeNew",	"normOld",	"normNew",	"normGen"])
m05 = DataFrame(CSV.File("max_05.csv", header=false))
rename!(m05, header)
show(m05, allrows=true, allcols=true)

50×10 DataFrame
 Row │ n        k        alpha       errOld       errNew       timeOld      timeNew      normOld   normNew   normGen  
     │ Float64  Float64  Float64     Float64      Float64      Float64      Float64      Float64   Float64   Float64  
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │    20.0      3.0   1.68642    2.21184e-10  5.18863e-13  0.00179469   0.000777386  0.574269  0.574269  0.574269
   2 │    95.0      9.0   0.0392704  9.16708e-12  9.3643e-14   0.00134563   0.000810651  0.435339  0.435339  0.435339
   3 │   818.0    631.0   0.213541   1.77093e-12  3.86826e-14  2.65707      1.69776      0.439288  0.439288  0.439288
   4 │   110.0     76.0   1.957      1.42964e-10  3.68103e-12  0.0445438    0.0275846    1.09439   1.09439   1.09439
   5 │    56.0     25.0  -0.826732   1.66782e-10  2.59465e-12  0.00598442   0.00443006   0.390783  0.390783  0.390783
   6 │   185.0     42.0  -0.0805532  1

In [18]:
describe(m05)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,n,195.32,4.0,48.0,991.0,0,Float64
2,k,90.48,2.0,15.5,631.0,0,Float64
3,alpha,0.753559,-0.826732,0.749545,2.06291,0,Float64
4,errOld,4.27916e-10,1.08961e-13,3.17228e-11,3.00824e-9,0,Float64
5,errNew,2.42586e-11,3.27537e-14,2.4049e-12,4.56195e-10,0,Float64
6,timeOld,0.240216,0.000586685,0.00460323,3.49317,0,Float64
7,timeNew,0.144006,0.000207682,0.0027298,2.05956,0,Float64
8,normOld,0.795729,0.187572,0.746433,1.55651,0,Float64
9,normNew,0.795729,0.187572,0.746433,1.55651,0,Float64


In [19]:
# same distance 0.5pi, but pretol=1e-5 -
header = vec(["n", "k",	"alpha",	"errOld",	"errNew",	"timeOld",	"timeNew",	"normOld",	"normNew",	"normGen"])
m05p5 = DataFrame(CSV.File("max_05_pretol5.csv", header=false))
rename!(m05p5, header)
describe(m05p5)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,n,195.32,4.0,48.0,991.0,0,Float64
2,k,90.48,2.0,15.5,631.0,0,Float64
3,alpha,0.753559,-0.826732,0.749545,2.06291,0,Float64
4,errOld,4.27916e-10,1.08961e-13,3.17228e-11,3.00824e-9,0,Float64
5,errNew,2.38998e-11,2.1155e-14,1.73786e-12,4.66694e-10,0,Float64
6,timeOld,0.241189,0.000591288,0.00471608,3.55047,0,Float64
7,timeNew,0.181335,0.000249446,0.00315943,2.66004,0,Float64
8,normOld,0.795729,0.187572,0.746433,1.55651,0,Float64
9,normNew,0.795729,0.187572,0.746433,1.55651,0,Float64


In [20]:
# distance at most 2pi
header = vec(["n", "k",	"alpha",	"errOld",	"errNew",	"timeOld",	"timeNew",	"normOld",	"normNew",	"normGen"])
m20 = DataFrame(CSV.File("max_20.csv", header=false))
rename!(m20, header)
describe(m20)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,n,195.32,4.0,48.0,991.0,0,Float64
2,k,90.48,2.0,15.5,631.0,0,Float64
3,alpha,0.753559,-0.826732,0.749545,2.06291,0,Float64
4,errOld,0.134594,6.6561e-13,4.54394e-10,1.61237,0,Float64
5,errNew,0.00528176,1.23292e-12,2.95398e-11,0.187642,0,Float64
6,timeOld,0.557739,0.000886699,0.0465941,8.11859,0,Float64
7,timeNew,0.347291,0.000352072,0.021976,5.28425,0,Float64
8,normOld,14.8671,0.678741,2.87488,421.71,0,Float64
9,normNew,2.74631,0.678741,2.69964,5.90342,0,Float64


In [21]:
# distance at most 4pi
header = vec(["n", "k",	"alpha",	"errOld",	"errNew",	"timeOld",	"timeNew",	"normOld",	"normNew",	"normGen"])
m40 = DataFrame(CSV.File("max_40.csv", header=false))
rename!(m40, header)
describe(m40)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,n,195.32,4.0,48.0,991.0,0,Float64
2,k,90.48,2.0,15.5,631.0,0,Float64
3,alpha,0.753559,-0.826732,0.749545,2.06291,0,Float64
4,errOld,0.298122,5.61147e-12,2.86051e-9,1.49669,0,Float64
5,errNew,0.004296,1.65024e-12,0.00031112,0.0973246,0,Float64
6,timeOld,1.75037,0.00166939,0.159145,19.7766,0,Float64
7,timeNew,1.11658,0.000616452,0.0552578,12.5347,0,Float64
8,normOld,255.505,1.13853,5.9207,3956.51,0,Float64
9,normNew,4.78403,1.08137,4.1965,11.8179,0,Float64


Displaying the full data table for $4\pi$. Here, there are several examples where log is beter than $f\pi|X|$.

In [22]:
show(m40, allrows=true, allcols=true)

50×10 DataFrame
 Row │ n        k        alpha       errOld       errNew       timeOld      timeNew       normOld     normNew   normGen  
     │ Float64  Float64  Float64     Float64      Float64      Float64      Float64       Float64     Float64   Float64  
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │    20.0      3.0   1.68642    0.876103     0.00843188    0.1343       0.0188507       8.50409   4.44371   4.59415
   2 │    95.0      9.0   0.0392704  1.32006e-9   7.21565e-10   0.00733889   0.00925292      3.48271   3.48271   3.48271
   3 │   818.0    631.0   0.213541   5.61147e-12  6.50801e-12   7.16874      3.80483         3.51431   3.51431   3.51431
   4 │   110.0     76.0   1.957      0.562042     0.00622488    6.90485      2.73808      3143.02      8.75221   8.75513
   5 │    56.0     25.0  -0.826732   1.82116e-9   6.80746e-11   0.0621894    0.026656        3.12626   3.12626   3.12626
   6 │   185.

# VERIFY FRECHET DERIVATIVE AND THE TRACE FORMULA

In [24]:
n = 5
A = 0.1*(reshape(0:(n*n-1), n, n)' .% 7) + UniformScaling(0.5)
E = reshape(0:(n*n-1), n, n)'
# E = E .* E
E = (E .* E) .% 23

e1 = exp(A)
dlt = 1e-8
e2 = exp(A + dlt*E)
println("VERIFYING FRECHET DERIVATIVE")
(e2-e1)/dlt
# expm_frechet_algo_64(A, E)[2]
println(linf((e2-e1)/dlt - Manifolds.expm_frechet(A, E)[2]))

println("VERIFYING THE TRACE FORMULA")
C = randn(n, n)
D = randn(n, n)
println(tr(C*Manifolds.expm_frechet(A, E)[2]*D))
println(tr(Manifolds.expm_frechet(A, D*C)[2]*E))

VERIFYING FRECHET DERIVATIVE
1.4476886789793753e-5
VERIFYING THE TRACE FORMULA
140.29041080348276
140.2904108034828
